### 기본 개념 : Multinomial classification and Softmax classification

* Logistic Regression의 경우, 실제 값(y_target)이 0, 1에 해당하는 값을 예측하는데, 실제 두개를 구분하는 hyper-plane을 찾는 것과 같은 것이다. (Binary Classification)
$$$$
* 이 아이디어를 그대로 Multinomial Classification에 적용할 수 있다. 즉 여러 개의 클래스가 존재하는, 예를 들어 여러 과목의 점수를 부여했을 때 학점 A, B, C의 결과 값을 산출하는 데이터의 경우, 각 클래스의 위치를 그래프에 그리면 각 클래스들을 구분하는 선을 찾는 문제가 된다.
$$$$
* 각 클래스를 구분하는 선을 찾는 방법 중 하나가, 각각의 클래스 두쌍을 구분하는 선들을 찾는 것이다. 예를 들어, A 학점과 B 학점을 구분하는 선, B학점과 C 학점을 구분하는 선, A 학점과 C학점을 구분하는 선인 Binary classification을 구하는 문제가 된다.
$$$$
* 이를 다시 정리하면, A 인지 아닌지, B 인지 아닌지, C 인지 아닌지를 구분하는 classifier를 찾는 것이다. 세번의 독립된 hypothesis를 구하는 것이나, 이것은 행렬을 이용하여 하나로 표현할 수 있다.


### sigmoid를 어떻게 적용할 것인가?  => softmax

* Multinomial Classification에서 clissifier(WX+b)를 적용하면 output은 0, 1이 아닌 벡터형태인 실수값이 나오게 되는데, 이 값들을 0~1 사이의 실수값이고 전체 합계가 1이 되도록 만들기 위하여 **softamx**를 적용한다. 


### Cost function : cross-entropy
$$ D(S,L) = -\sum L_i log(S_i)$$

S는 예측값([0.7, 2.0, 0.1]), L은 실제 값([1.0, 0.0, 0.0])이다.

* cost 함수를 보면, 예측 값을 로그 값으로 변환하여 실제 값과 곱한 후 마이너스 부호로 바꾼 것이다. 이 함수는 예측이 맞았을 때 0, 예측이 틀렸을 때는 무한대로 수렴한다.
$$$$
* 실제 Logistic cost 함수는 cross entropy cost 함수에 해당한다.

* 위의 함수는 하나의 classifier에 해당하는 것이고, 전체 적용하면, 
$$Loss = \dfrac{1}{N} \sum_i D(S(Wx_i+b), L_i)$$

이 된다.

여기에서는 i 는 training set

### Gradient descent Algorithm
마찬가지로 위의 Loss function을 미분한 후 weight 값을 찾는다.

### softmax function의 tensorflow 구현

tf.matmul(X, W) + b의 결과인 scores(Logit)를 확률 값으로 변환시켜 준다.

In [1]:
import tensorflow as tf

In [2]:
x_data = [[1, 2, 1, 1], [2, 1, 3, 2], [3, 1, 3, 4], [4, 1, 5, 5], [1, 7, 5, 5], [1, 2, 5, 6], [1, 6, 6, 6], [1, 7, 7, 7]]
y_data = [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0]]

In [3]:
# placeholders
X = tf.placeholder(tf.float32, shape=[None, 4])
Y = tf.placeholder(tf.float32, shape=[None, 3])
nb_classes = 3

# variables
W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# model : softmax = exp(logits) / reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# cross entropy cost/loss function
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Launch Graph
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(2001):
    sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
    if step % 200 == 0:
        print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
0 0.9386492
200 0.60085815
400 0.49826515
600 0.40517354
800 0.31460842
1000 0.24741668
1200 0.22371906
1400 0.20408642
1600 0.18748231
1800 0.17326656
2000 0.16096872


cost 함수 값은 0.93에서 0.16으로 감소하였다.

In [6]:
# 예측
a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})  # softmax 함수 값을 거친 확률 값을 반환한다.
print(a, sess.run(tf.argmax(a, 1)))   # tf.argmax는 one-hot encoding

[[2.9869851e-02 9.7012526e-01 4.9022365e-06]] [1]


In [5]:
a.sum()

1.0

1번째 클래스가 거의 0.97로 클래스 1에 해당한다.

In [9]:
# 여러개를 예측할 때
all = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]]})
print(all, sess.run(tf.argmax(all, 1)))

[[2.9869851e-02 9.7012526e-01 4.9022369e-06]
 [6.8856853e-01 3.0117500e-01 1.0256412e-02]
 [9.2143120e-09 2.8390205e-04 9.9971610e-01]] [1 0 2]
